In [ ]:
!wget -O 'dataset' 'https://storage.googleapis.com/kaggle-data-sets/111880%2F269359%2Fbundle%2Farchive.zip?GoogleAccessId=gcp-kaggle-com@kaggle-161607.iam.gserviceaccount.com&Expires=1593864510&Signature=VI7UzqQr9WUFt6tPspjaciQ0rvWhOnzBd02wN02BcSTRafyjSTW6JtV5GOy9A8Ro1llbalKOvPt%2FfIOFdc4Kus6GNuJ%2FtGWMlMT%2BUKYjrvVOMfG3rFSk0rqyfz5oD4bkCYNZUfdK2xjsK4LGisHH7JtK4Pey2wgREpIqCXcHq3H0etyzwalxMbY2zogpk%2Bq5vXzOFZRj%2BX1BKbFz7%2Bpb0vqK6eWVqPyhZD4VUQQ%2FNocqVuX4BQpSFiyx6X0tEyx8cXSaVIWrKwzRv9CsVV4dTrIp775IjkFj0EIKQwox5CeLc9fE3zvx%2BccNodZ0aW1lips%2BHmtJu6n3PovPSfIfJw%3D%3D'

# !wget -O 'dataset' 'https://storage.googleapis.com/kaggle-data-sets/137362%2F325302%2Fbundle%2Farchive.zip?GoogleAccessId=gcp-kaggle-com@kaggle-161607.iam.gserviceaccount.com&Expires=1593864065&Signature=BB9LoFIi0IcMyKUVokgqtCawp0iBKiId0EM6O6iABczGy5E9Vru7V5AMTEfYz%2Bfhks6MuChZVk1qk%2BD%2BnNaZemqLHpsUWMATh2a6GWE3lcx00kWXv160euEFfjCTUvEJl%2F7KEI%2FEQAekoV6Kdvwen57%2Fp5UFS%2BYUVA75DftxxLlJwyV4O9otAH1peYIlNIWWcmD%2B8CxfbaHEiLbnF5U9me7mkFYP3c2GqXEshbRWSps0EezuJCgSaCD%2B3ePsMKNfd9jdykdSKrX8gwNl%2FTw6bxKyH%2F%2By4Dm%2BmMD7m4bqlRkthyGMa5lNTnyIwbFdReWnIGKY98%2BOaaf0IZb317%2FJBw%3D%3D'


--2020-07-01 13:35:19--  https://storage.googleapis.com/kaggle-data-sets/111880%2F269359%2Fbundle%2Farchive.zip?GoogleAccessId=gcp-kaggle-com@kaggle-161607.iam.gserviceaccount.com&Expires=1593864510&Signature=VI7UzqQr9WUFt6tPspjaciQ0rvWhOnzBd02wN02BcSTRafyjSTW6JtV5GOy9A8Ro1llbalKOvPt%2FfIOFdc4Kus6GNuJ%2FtGWMlMT%2BUKYjrvVOMfG3rFSk0rqyfz5oD4bkCYNZUfdK2xjsK4LGisHH7JtK4Pey2wgREpIqCXcHq3H0etyzwalxMbY2zogpk%2Bq5vXzOFZRj%2BX1BKbFz7%2Bpb0vqK6eWVqPyhZD4VUQQ%2FNocqVuX4BQpSFiyx6X0tEyx8cXSaVIWrKwzRv9CsVV4dTrIp775IjkFj0EIKQwox5CeLc9fE3zvx%2BccNodZ0aW1lips%2BHmtJu6n3PovPSfIfJw%3D%3D
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.133.128, 108.177.15.128, 173.194.76.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.133.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 363152213 (346M) [application/zip]
Saving to: ‘dataset’

dataset             100%[===================>] 346.33M  50.8MB/s    in 6.8s    

2020-07-01 13:35:27 (5

In [ ]:
src_zip = 'dataset'
cpy_zip = 'dataset_folder/' 

import zipfile
with zipfile.ZipFile(src_zip, 'r') as zip_ref:
    zip_ref.extractall(cpy_zip)


In [ ]:
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
from glob import glob
import random

from keras.utils.np_utils import to_categorical


Using TensorFlow backend.


In [ ]:
def make_lab_dict(path_dataset):
    folders = os.listdir(path_dataset)
    folders = np.sort(folders)  
    labs = list(range(0,len(folders)))
    lab_dict = dict(zip(folders, labs))
    
    return lab_dict


In [ ]:
from tensorflow.keras.layers import Input, Dropout, Dense
from tensorflow.keras.layers import BatchNormalization, Flatten, Lambda
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Conv2DTranspose, Conv2D
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.regularizers import l2
from tensorflow.keras import Sequential
from tensorflow.keras.models import Model
import tensorflow as tf


CHANNELS = 3
#SCALED_Y,SCALED_X = (128,128)


def build_encoder(in_img, filters = 32):
    def conv2d(layer_input, filters, f_size=4, bn=True):
        d = Conv2D(filters, kernel_size=f_size,
                   strides=(2,2), padding='same')(layer_input)
        
        if bn:
            d = BatchNormalization(momentum=0.8)(d)
        d = LeakyReLU(alpha=0.2)(d)

        return d
    
    #d0 = Input(shape=img_shape)    
    d0_b = Conv2D(filters, kernel_size=4,strides=1, padding='same', activation='relu')(in_img)
  
    # Downsampling: 7 x stride of 2 --> x1/128 downsampling
    d1 = conv2d(d0_b, filters, bn=False)
    d2 = conv2d(d1, filters * 2)
    d3 = conv2d(d2, filters * 4)
    d4 = conv2d(d3, filters * 8)
    d5 = conv2d(d4, filters * 8)
    d6 = conv2d(d5, filters * 8)
    d7 = conv2d(d6, filters * 8)
    
    return d7

def build_decoder(input_features,channels=CHANNELS,filters = 32, output_activation = 'sigmoid'):
    
    def deconv2d(layer_input, filters, f_size=4, dropout_rate=0):
        
        u = Conv2DTranspose(filters, kernel_size=f_size, strides=(2,2),
                            padding='same', activation='linear')(layer_input)
        u = Conv2D(filters, kernel_size=f_size, strides=1,
                   padding='same',activation='relu')(u)


        u = BatchNormalization(momentum=0.8)(u)
        if dropout_rate:
            u = Dropout(dropout_rate)(u)
        
        return u
    
    #input_features = Input(shape=feat_shape) 
    # Upsampling: 6 x stride of 2 --> x64 upsampling 
    u1 = deconv2d(input_features ,filters * 8)
    u2 = deconv2d(u1, filters * 8)
    u3 = deconv2d(u2, filters * 8)
    u4 = deconv2d(u3, filters * 4)
    u5 = deconv2d(u4, filters * 2)
    u6 = deconv2d(u5, filters)
    
    u7 = deconv2d(u6, filters)
    
    # added conv layers after the deconvs to avoid the pixelated outputs    
    output_img = Conv2D(channels, kernel_size=4,
                        strides=1, padding='same',
                        activation=output_activation)(u7)
    
    return output_img


In [ ]:
def get_paths(path_dataset):
    all_paths = glob(path_dataset+'/'+'*'+'/'+'*')
    return all_paths

In [ ]:
def my_datagen(path_dataset, lab_dict, batch_size=64, out_shape=(128,128,3), mode='auto'):
    
    while True: 
        all_paths = get_paths(path_dataset)
        selected_paths = random.sample(all_paths, batch_size)
        
        batch_images = []
        batch_labels = []
        for i in range(0,len(selected_paths)):
            img = cv2.imread(selected_paths[i])
            img = cv2.resize(img,(out_shape[0],out_shape[1]))

            img_category = selected_paths[i].split('/')[-2]
            lab = lab_dict[img_category]
            lab = to_categorical(lab, num_classes=6)
            # print(img_category)
            # print(lab)
            batch_images.append(img)
            batch_labels.append(lab)
        if mode=='auto':
            yield np.array(batch_images)/255, np.array(batch_images)/255
        elif mode=='classi':
            yield np.array(batch_images)/255, np.array(batch_labels)
        else:
            yield np.array(batch_images)/255, (np.array(batch_images)/255,np.array(batch_labels))

In [ ]:
def build_multitask_model(input_shape):
    input_img = Input(shape = input_shape)
    enc = build_encoder(input_img)

    x = Flatten()(enc)
    x = Dense(256, activation='relu')(x)
    x = Dense(6,activation='softmax')(x)

    y = build_decoder(enc)

    multi_model = Model(input_img, [y,x])

    return multi_model

In [ ]:
def init_model(model_name='my_model', path_ckpt='checkpoint/', input_shape = (128,128,3),mode='auto'):
    complete_path = path_ckpt+model_name+'.h5'

    if mode=='auto':    
        input_img = Input(shape = input_shape)
        model = Model(input_img, build_decoder(build_encoder(input_img)))
    else:
        model = build_multitask_model(input_shape=input_shape)
        
    if not os.path.isdir(path_ckpt):
        os.mkdir(path_ckpt)

    names = os.listdir(path_ckpt)
    if complete_path.split('/')[-1] in names:
        print('Loading checkpoint...')
        model.load_weights(complete_path)
    else:
        print('Training from scratch')

    return model, complete_path

    

In [ ]:
def train_model(path_dataset, batch_size=64, epochs=2, img_shape=(128,128,3), mode='auto'):
    model, checkpoint_path = init_model(input_shape=img_shape, mode=mode)
    
    checkpoint=ModelCheckpoint(filepath=checkpoint_path,monitor='loss',save_best_only=True,
                           save_weights_only=False,verbose=0,mode="auto")
    
    # checkpoint = ModelCheckpoint(filepath=checkpoint_path, monitor='loss', save_best_only=False, 
    #                              save_weights_only=False, mode='auto', period=2)

    callbacks=[checkpoint]
    print('learning rate = ',learn_rate)
    opt = tf.keras.optimizers.Adam(lr = learn_rate)
    if mode=='auto': 
        model.compile(optimizer=opt,loss="mse")
    else:
        model.compile(optimizer=opt,loss=["mse", "categorical_crossentropy"])
    
    lab_dict = make_lab_dict(path_dataset)
    print(lab_dict)

    data_size = len(get_paths(path_dataset))

    train_gen = my_datagen(path_dataset, lab_dict, batch_size=batch_size, out_shape=img_shape, mode=mode)
    model.fit_generator(train_gen, epochs=epochs, callbacks = callbacks,steps_per_epoch=int(data_size/batch_size))

    # save trained model
    #model.save('checkpoint/' + 'trained_model.h5')


In [ ]:
global learn_rate
learn_rate = 0.0001

batch_size = 256
input_shape = (128,128,3)
path_dataset = 'dataset_folder/seg_train/seg_train/'

In [ ]:
train_model(path_dataset, batch_size, 300, input_shape, mode='multi')


Training from scratch
learning rate =  0.0001
{'buildings': 0, 'forest': 1, 'glacier': 2, 'mountain': 3, 'sea': 4, 'street': 5}
Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/300
36/54 [===================>..........] - ETA: 30s - loss: 1.1555 - conv2d_15_loss: 0.0828 - dense_1_loss: 1.0727